### Data Exploration and Preprocessing

#### Initialise Spark Session and Load Data from HDFS into Spark DataFrames

In [ ]:
from pyspark.sql import SparkSession

spark_local = SparkSession.builder.appName("MovieLensAnalysisLocal").getOrCreate()

movies_df = spark_local.read.csv("hdfs://namenode:9000/movies.csv", header=True, inferSchema=True)
ratings_df = spark_local.read.csv("hdfs://namenode:9000/ratings.csv", header=True, inferSchema=True)
tags_df = spark_local.read.csv("hdfs://namenode:9000/tags.csv", header=True, inferSchema=True)
links_df = spark_local.read.csv("hdfs://namenode:9000/links.csv", header=True, inferSchema=True)

#### Explore Data Schema and Sample Records

In [ ]:
movies_df.printSchema()
movies_df.show(5)

ratings_df.printSchema()
ratings_df.show(5)

tags_df.printSchema()
tags_df.show(5)

links_df.printSchema()
links_df.show(5)

#### Handle Missing Values and Duplicates

In [ ]:
movies_df = movies_df.na.drop()
ratings_df = ratings_df.na.drop()
tags_df = tags_df.na.drop()
links_df = links_df.na.drop()

movies_df = movies_df.dropDuplicates()
ratings_df = ratings_df.dropDuplicates()
tags_df = tags_df.dropDuplicates()
links_df = links_df.dropDuplicates()

#### Split Genres into array

In [ ]:
from pyspark.sql.functions import split

movies_df = movies_df.withColumn("genres_array", split(movies_df["genres"], "\\|"))

### CRUD Operations
#### Create

In [ ]:
from pyspark.sql import Row
from pyspark.sql.functions import split

movies_df_copy = movies_df


new_movie = Row(movieId=999999, title="New Movie (2024)", genres="Sci-Fi|Drama")


new_movie_df = spark_local.createDataFrame([new_movie])
new_movie_df = new_movie_df.withColumn("genres_array", split(new_movie_df["genres"], "\\|"))
movies_df_copy = movies_df_copy.select("movieId", "title", "genres").withColumn("genres_array", split(movies_df_copy["genres"], "\\|"))
movies_df_copy = movies_df_copy.union(new_movie_df.select("movieId", "title", "genres", "genres_array"))
movies_df_copy.filter(movies_df_copy.movieId == 999999).show()

#### Read

In [ ]:
from pyspark.sql.functions import regexp_extract, col

# filter movies by sci-fi genre
sci_fi_movies = movies_df_copy.filter(
    (movies_df.genres.contains("Sci-Fi"))
)

# select and display the top 5 movies
sci_fi_movies.select("title", "genres").show(5, truncate=False)

#### Update

In [ ]:
from pyspark.sql.functions import when

# update the genres for the movie with movieId = 999999 (one I created ealier)
movies_df_copy = movies_df_copy.withColumn(
    "genres",
    when(movies_df_copy.movieId == 999999, "Thriller|Action").otherwise(movies_df_copy.genres)
)

# Verify the update
movies_df_copy.filter(movies_df_copy.movieId == 999999).select("movieId", "title", "genres").show()

#### Delete

In [ ]:
from pyspark.sql.functions import regexp_extract

movies_df_copy = movies_df_copy.withColumn("year", regexp_extract(movies_df_copy["title"], r"\((\d{4})\)", 1))
movies_df_copy = movies_df_copy.filter(movies_df_copy.year != "2020")

movies_df_copy.write.mode("overwrite").parquet("hdfs://namenode:9000/movies_df_copy")

### Now use master for complex SQL queries

In [ ]:
spark_master = SparkSession.builder \
        .appName("MovieLensAnalysis") \
        .master("spark://spark-master:7077") \
        .getOrCreate()

movies_df = spark_master.read.csv("hdfs://namenode:9000/movies.csv", header=True, inferSchema=True)
ratings_df = spark_master.read.csv("hdfs://namenode:9000/ratings.csv", header=True, inferSchema=True)
tags_df = spark_master.read.csv("hdfs://namenode:9000/tags.csv", header=True, inferSchema=True)
links_df = spark_master.read.csv("hdfs://namenode:9000/links.csv", header=True, inferSchema=True)

movies_df = movies_df.na.drop()
ratings_df = ratings_df.na.drop()
tags_df = tags_df.na.drop()
links_df = links_df.na.drop()
movies_df = movies_df.dropDuplicates()
ratings_df = ratings_df.dropDuplicates()
tags_df = tags_df.dropDuplicates()
links_df = links_df.dropDuplicates()

movies_df = movies_df.withColumn("genres_array", split(movies_df["genres"], "\\|"))

### Complex Big Data Queries Using Spark SQL

In [ ]:
movies_df.createOrReplaceTempView("movies")
ratings_df.createOrReplaceTempView("ratings")
tags_df.createOrReplaceTempView("tags")
links_df.createOrReplaceTempView("links")

#### Query 1: Top Tags Associated with High-Rated Movies

In [ ]:
import matplotlib.pyplot as plt
top_tags = spark_master.sql("""
SELECT
    t.tag,
    COUNT(*) AS tag_count
FROM tags t
JOIN (
    SELECT
        movieId,
        AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY movieId
    HAVING avg_rating > 4.5
) high_rated_movies ON t.movieId = high_rated_movies.movieId
GROUP BY t.tag
ORDER BY tag_count DESC
LIMIT 10
""")

top_tags_pd = top_tags.toPandas()
plt.barh(top_tags_pd['tag'], top_tags_pd['tag_count'])
plt.xlabel('Tag Count')
plt.ylabel('Tag')
plt.title('Top Tags for High-Rated Movies')
plt.gca().invert_yaxis()
plt.show()

#### Query 2: Top Genres by Average Rating

In [ ]:
top_genres = spark_master.sql("""
SELECT
    genre,
    AVG(avg_rating) AS avg_genre_rating
FROM (
    SELECT
        genre,
        AVG(r.rating) AS avg_rating
    FROM movies m
    JOIN ratings r ON m.movieId = r.movieId
    LATERAL VIEW explode(m.genres_array) AS genre
    GROUP BY genre
) tmp
GROUP BY genre
ORDER BY avg_genre_rating DESC
LIMIT 5
""")

top_genres_pd = top_genres.toPandas()
plt.bar(top_genres_pd['genre'], top_genres_pd['avg_genre_rating'])
plt.xlabel('Genre')
plt.ylabel('Average Rating')
plt.title('Top 5 Genres by Average Rating')
plt.show()

####  Query 3: Correlation Between Movie Popularity and Average Rating

In [ ]:
popularity_vs_rating = spark_master.sql("""
SELECT
    m.movieId,
    m.title,
    COUNT(r.rating) AS num_ratings,
    AVG(r.rating) AS avg_rating
FROM movies m
JOIN ratings r ON m.movieId = r.movieId
GROUP BY m.movieId, m.title
HAVING COUNT(r.rating) > 100
ORDER BY num_ratings DESC
""")

popularity_vs_rating_pd = popularity_vs_rating.toPandas()
plt.scatter(popularity_vs_rating_pd['num_ratings'], popularity_vs_rating_pd['avg_rating'])
plt.xlabel('Number of Ratings')
plt.ylabel('Average Rating')
plt.title('Movie Popularity vs. Average Rating')
plt.show()

#### Query 4: Most Active Users and the Movies They Rated the Most

In [ ]:
most_active_users_avg = spark_master.sql("""
SELECT
    r.userId,
    COUNT(r.movieId) AS movie_count,
    AVG(r.rating) AS avg_rating
FROM ratings r
JOIN movies m ON r.movieId = m.movieId
GROUP BY r.userId
ORDER BY movie_count DESC
LIMIT 10
""")

import matplotlib.pyplot as plt

most_active_users_avg_pd = most_active_users_avg.toPandas()
plt.figure(figsize=(10,6))
plt.barh(most_active_users_avg_pd['userId'].astype(str), 
         most_active_users_avg_pd['avg_rating'])
plt.xlabel('Average Rating')
plt.ylabel('User ID')
plt.title('Average Rating Given by Top 10 Most Active Users')
plt.gca().invert_yaxis()
plt.show()